In [2]:
import hoomd
import hoomd.md

HOOMD-blue v2.1.3 CUDA (8.0) DOUBLE HPMC_MIXED MPI SSE SSE2 SSE3 SSE4_1 SSE4_2 AVX 
Compiled: 02/07/2017
Copyright 2009-2016 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----


# Advancing simulations forward in time

Use `hoomd.run` to advance a simulation forward a given number of time steps. Start with the Lennard-Jones tutorial script.

In [3]:
hoomd.context.initialize('--mode=cpu')
system = hoomd.init.create_lattice(unitcell=hoomd.lattice.sc(a=2.0), n=5)
nl = hoomd.md.nlist.cell()
lj = hoomd.md.pair.lj(r_cut=3.0, nlist=nl)
lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0)
all = hoomd.group.all();
hoomd.md.integrate.mode_standard(dt=0.001)
hoomd.md.integrate.brownian(group=all, kT=0.1, seed=987)

HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 125 particles
notice(2): integrate.langevin/bd is using specified gamma values


After initialization with `create_lattice`, the simulation starts at time step 0. Query the current step counter with ``get_step``.

In [4]:
hoomd.get_step()

0

Each HOOMD script can have one **integrator** active at a time. This example uses the `mode_standard` integrator from the `md` package. On each time step, `mode_standard` advances the simulation forward `dt` time units, integrating the equations of motion specified in the provided integration methods (brownian dynamics in this example). Use `hoomd.run` to advance the simulation a given number of steps.

In [5]:
hoomd.run(1000)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 125
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:32 | Step 1000 / 1000 | TPS 1649.81 | ETA 00:00:00
Average TPS: 1645.57
---------
-- Neighborlist stats:
54 normal updates / 11 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 22 / n_neigh_avg: 9.392
shortest rebuild period: 12
-- Cell list stats:
Dimension: 2, 2, 2
n_min    : 11 / n_max: 19 / n_avg: 15.625
** run complete **


Now we are at step 1000.

In [6]:
hoomd.get_step()

1000

## Run options

You may want to supress it with the `quiet` option.

In [8]:
hoomd.run(1000, quiet=True)
hoomd.get_step()

3000

You may want to limit a run to a given length of time (e.g. for short benchmark runs). Use `limit_hours` to stop a run after so many hours, even if it has not reached the requested number of time steps.

In [9]:
hoomd.run(1e6, limit_hours=5/3600, quiet=True)

notice(2): Ending run at time step 51329 as 0.00138889 hours have passed


The profile option prints information on how much time was spent in each portion of the calculation. This is helpful when trying to identify why a simulation is running slowly.

In [10]:
hoomd.run(10000, profile=True)

** starting run **
Time 00:01:09 | Step 61329 / 61329 | TPS 8140.78 | ETA 00:00:00
Average TPS: 8133.16
Simulation:  1.2296s | 100.000% 
        Integrate:  0.3634s | 29.558% 
                BD step 1:  0.3354s | 27.278% 
                Net force:  0.0209s |  1.700% 
                Sum accel:  0.0000s |  0.000% 
                Self:       0.0071s |  0.580% 
        Neighbor:   0.1113s |  9.051% 
                Cell:        0.0032s |  0.259% 
                        compute:  0.0028s |  0.230% 
                        init:     0.0000s |  0.003% 
                Dist check:  0.0125s |  1.016% 
                compute:     0.0927s |  7.540% 
                head-list:   0.0001s |  0.005% 
                Self:        0.0028s |  0.231% 
        Pair lj:    0.7449s | 60.580% 
        SFCPack:    0.0026s |  0.208% 
        Self:       0.0074s |  0.604% 
---------
-- Neighborlist stats:
264 normal updates / 101 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 54 / n_ne

## Callbacks

You can specify an arbitrary python callback to run. This function will be called at the given period. If the function returns a negative value, the run is aborted.

In [11]:
def my_callback(step):
    print("This is step", step)
    if step >= 70000:
        return -1
    else:
        return 0

hoomd.run(1e6, callback=my_callback, callback_period=1000, quiet=True)

This is step 62000
This is step 63000
This is step 64000
This is step 65000
This is step 66000
This is step 67000
This is step 68000
This is step 69000
This is step 70000
notice(2): End of run requested by python callback at step 70000 / 1061329
